# Topology Estimation Model

This notebook contains all the steps of topology model developement. 

In this notebook, the model can be:
- trained and tested
- loaded and run for new data

In [1]:
import sys
import os
import torch

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
print("Starting run from main script...")

Starting run from main script...


## 1. Load dataset

In [3]:
from data.prep import DataPreprocessor
from data.config import DataConfig

data_config = DataConfig(run_type='train')
data_preprocessor = DataPreprocessor(package='topology_estimation')

# load datalaoders
train_package, test_package, val_package = data_preprocessor.get_training_data_package(data_config, train_rt=0.8, test_rt=0.1, val_rt=0.1, batch_size=50)

train_loader, train_data_stats = train_package
test_loader, test_data_stats = test_package
val_loader, val_data_stats = val_package


'Train' type dataset selected:


ds_subtype selections:

(<ds_subtype_num>) <ds_subtype> : [<augments>]
---------------------------------------------
>> Healthy configs
(1) series_tp_(fs=1000)    : [OG]

>> Unhealthy configs

>> Unknown configs


Node and signal types are set as follows:

(<node_num>) <node_type> : [<signal_types>]
---------------------------------------------
(1) node_group_name   : [m, 0, 0, 4]
(2) group   : [mass_1, mass_2, mass_3, mass_4]


For ds_type 'OK' and others....

Maximum timesteps across all node types: 100001

'fs' is updated in data_config as given in loaded healthy (or unknown) data.
New fs:
[[1000., 1000., 1000.],
 [1000., 1000., 1000.],
 [1000., 1000., 1000.],
 [1000., 1000., 1000.]]

No exclusive rep numbers found in keys of hfd5 file. Hence, using default rep numbers.


[1 sample = (n_nodes, n_timesteps (window_length), n_dims)]
---------------------------------------------
Total samples: 1000 
Train: 800/800 [OK=800, NOK=0, UK=0], Test: 100/100 [

#### Set the number of timesteps and dimensions of the node data

In [4]:
dataiter = iter(train_loader)
data = next(dataiter)

n_nodes = data[0].shape[1]
n_timesteps = data[0].shape[2]
n_dims = data[0].shape[3]

print(f"Number of nodes: {n_nodes}")
print(f"Number of timesteps: {n_timesteps}")  
print(f"Number of dimensions: {n_dims}")

Number of nodes: 4
Number of timesteps: 100
Number of dimensions: 3


In [5]:
data[1][0]

tensor([1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1.])

### Prepare the relation matrix for encoder input

##### Generate off-diagonal fully connected graph

In [6]:
from settings.manager import NRITrainManager
from topology_estimation.relations import RelationMatrixMaker

nri_config = NRITrainManager(data_config)

rm = RelationMatrixMaker(nri_config.spf_config)
rec_rel, send_rel = rm.get_relation_matrix(train_loader)



Loading Relation Matrices...

Reciever relation matrix:
tensor([[0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.]]) 
shape: torch.Size([12, 4])

Sender relation matrix:
tensor([[1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.]]) 
shape: torch.Size([12, 4])

---------------------------------------------------------------------------


## 2. Load and prepare the topology estimator model blocks

### Prepare params

In [7]:
from encoder import Encoder
from decoder import Decoder
from torchinfo import summary
import inspect

def get_encoder_params(tp_config, train_loader):
    """
    Get the encoder parameters required for initializing the NRI model.

    Returns
    -------
    enc_model_params : dict
        Dictionary containing the encoder model parameters.
    enc_run_params : dict
        Dictionary containing the encoder run parameters.
    """
    # encoder params
    req_enc_model_params = inspect.signature(Encoder.__init__).parameters.keys()
    req_enc_run_params = inspect.signature(Encoder.set_run_params).parameters.keys()

    enc_model_params = {
        key.removesuffix('_enc'): value for key, value in tp_config.__dict__.items() if key.removesuffix('_enc') in req_enc_model_params and key not in ['hyperparams']
    }
    enc_run_params = {
        key.removeprefix('enc_'): value for key, value in tp_config.__dict__.items() if key.removeprefix('enc_') in req_enc_run_params and key not in ['data_config']
    }
    # get n_comps and n_dims for encoder and decoder
    none_dict = {param: None for param in inspect.signature(Encoder).parameters.keys()}
    pre_enc = Encoder(**none_dict)
    pre_enc.set_run_params(**enc_run_params)
    n_comps, n_dims = pre_enc.process_input_data(next(iter(train_loader))[0], get_data_shape=True)

    enc_model_params['n_comps'] = n_comps
    enc_model_params['n_dims'] = n_dims

    print("\n<<<<< ENCODER PARAMETERS >>>>>")
    print("Encoder model parameters:")
    print(15 * "-")
    for key, value in enc_model_params.items():
        print(f"{key}: {value}")

    print("\nEncoder run parameters:")
    print(15 * "-")
    for key, value in enc_run_params.items():
        print(f"{key}: {value}")

    print('\n' + 35*'-')

def get_decoder_params(tp_config, train_loader):
    """
    Get the decoder parameters
    """
    # decoder params
    req_dec_model_params = [param for param in Decoder().__dict__.keys()] 
    req_dec_run_params = inspect.signature(Decoder.set_run_params).parameters.keys()

    dec_model_params = {
        key.removesuffix('_dec'): value for key, value in tp_config.__dict__.items() if key.removesuffix('_dec') in req_dec_model_params and key not in ['hyperparams']
    }
    dec_run_params = {
        key.removeprefix('dec_'): value for key, value in tp_config.__dict__.items() if key.removeprefix('dec_') in req_dec_run_params and key not in ['data_config']
    }
    dec_model_params['n_dims'] = next(iter(train_loader))[0].shape[3]

    print("\n<<<<< DECODER PARAMETERS >>>>>")
    print("\nDecoder model parameters:")
    print(15 * "-")
    for key, value in dec_model_params.items():
        print(f"{key}: {value}")

    print("\nDecoder run parameters:")
    print(15 * "-")
    for key, value in dec_run_params.items():
        print(f"{key}: {value}")

    print('\n' + 35*'-')

    return dec_model_params, dec_run_params


In [8]:
# enc_model_params, enc_run_params = get_encoder_params()
# dec_model_params, dec_run_params = get_decoder_params()

### Make NRI model

In [9]:
# from nri import NRI

# nri_model = NRI(enc_model_params, dec_model_params)
# nri_model.set_run_params(enc_run_params, dec_run_params, nri_config.temp, nri_config.is_hard)


# print("\nNRI Model Summary:")
# # print(summary(nri_model, (train_loader.batch_size, n_nodes, n_timesteps, n_dims)))
# print(nri_model)

In [10]:
# from utils.custom_loader import CombinedDataLoader

# train_loader_full = CombinedDataLoader(train_loader, rel_loader)

## 3. Training NRI model

In [11]:
# from pytorch_lightning import Trainer
# from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.callbacks import RichProgressBar
# from utils.custom_loader import CombinedDataLoader

# nri_model.set_training_params()

# # get log path
# train_log_path = nri_config.get_train_log_path(n_comps=enc_model_params['n_comps'], n_dim=dec_model_params['n_dims'])

# if nri_config.is_log:
#     nri_config.save_params()
#     logger = TensorBoardLogger(os.path.dirname(train_log_path), name="", version=os.path.basename(train_log_path))


# trainer = Trainer(
#     max_epochs=nri_config.max_epochs,
#     logger=logger,
#     enable_progress_bar=True,
#     log_every_n_steps=1,)

# trainer.fit(model=nri_model, train_dataloaders=CombinedDataLoader(train_loader, rel_loader))

## Decoder

### Prep params

In [12]:
from settings.manager import DecoderTrainManager
from decoder import Decoder
from torchinfo import summary
import inspect

dec_config = DecoderTrainManager(data_config)
dec_model_params, dec_run_params = get_decoder_params(dec_config, train_loader)


<<<<< DECODER PARAMETERS >>>>>

Decoder model parameters:
---------------
n_edge_types: 1
msg_out_size: 64
edge_mlp_config: [[64, 'tanh'], [32, 'tanh'], [16, 'tanh'], [64, None]]
out_mlp_config: [[64, 'tanh'], [32, 'tanh'], [16, 'tanh'], [64, None]]
do_prob: 0
is_batch_norm: True
recur_emb_type: gru
n_dims: 3

Decoder run parameters:
---------------
domain_config: {'type': 'time', 'cutoff_freq': 0}
raw_data_norm: None
feat_configs: []
reduc_config: None
feat_norm: None
skip_first_edge_type: False
pred_steps: 1
is_burn_in: False
burn_in_steps: 1
is_dynamic_graph: False
temp: 1.0
is_hard: True

-----------------------------------


### Make Deocder model

In [13]:
from topology_estimation.decoder import Decoder

# prep hyperparams
dec_config.hyperparams.update({
    'n_dims': str(dec_model_params['n_dims']),
    'n_nodes': str(next(iter(train_loader))[0].shape[1])  
    })

decoder_model = Decoder()
for key, value in dec_model_params.items():
    setattr(decoder_model, key, value)

decoder_model.hyperparams = dec_config.hyperparams
decoder_model.build_model()

decoder_model.set_input_graph(rec_rel, send_rel)

edge_matrix = 0.5*(rec_rel + send_rel).sum(dim=-1, keepdim=True)
edge_matrix = edge_matrix.unsqueeze(0).repeat(train_loader.batch_size, 1, 1)

decoder_model.set_edge_matrix(edge_matrix)

decoder_model.set_run_params(
    **dec_run_params, 
    data_config=data_config, 
    data_stats=train_data_stats
    )

decoder_model.set_training_params(
    lr=dec_config.lr, 
    optimizer=dec_config.optimizer,
    loss_type=dec_config.loss_type
    )

print("\n" + 75*'-')
print("\nDecoder Model Initialized with the following configurations:")
print("\nDecoder Model Summary:")
print(decoder_model)
# print(summary(decoder_model, (next(iter(train_loader))[0].shape), device=device))

print('\n' + 75*'-')


---------------------------------------------------------------------------

Decoder Model Initialized with the following configurations:

Decoder Model Summary:
Decoder(
  (edge_mlp_fn): ModuleList(
    (0): MLP(
      (layers): ModuleList(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Tanh()
        (3): Dropout(p=0, inplace=False)
        (4): Linear(in_features=64, out_features=32, bias=True)
        (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): Tanh()
        (7): Dropout(p=0, inplace=False)
        (8): Linear(in_features=32, out_features=16, bias=True)
        (9): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): Tanh()
        (11): Dropout(p=0, inplace=False)
        (12): Linear(in_features=16, out_features=64, bias=True)
        (13): BatchNorm1d(64, eps=1

## Train Decoder

In [14]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import RichProgressBar
from utils.custom_loader import CombinedDataLoader

decoder_model.set_training_params()

# get log path
train_log_path = dec_config.get_train_log_path(n_dim=dec_model_params['n_dims'])

if dec_config.is_log:
    dec_config.save_params()
    logger = TensorBoardLogger(os.path.dirname(train_log_path), name="", version=os.path.basename(train_log_path))

trainer = Trainer(
    max_epochs=dec_config.max_epochs,
    logger=logger,
    enable_progress_bar=True,
    log_every_n_steps=1,
    num_sanity_val_steps=0)

trainer.fit(model=decoder_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


'(m004)-(gru)_decoder_1.1' already exists in the log path 'c:\Aryan_Savant\Thesis_Projects\my_work\AFD_thesis\topology_estimation\logs\mass_sp_dm\M004\scene_1\decoder\train\etypes=1\g=m004\D=gru\(m004)-(gru)_decoder_1.1'.


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type       | Params | Mode 
---------------------------------------------------------
0 | edge_mlp_fn       | ModuleList | 12.3 K | train
1 | recurrent_emb_fn  | GRU        | 13.2 K | train
2 | mean_mlp          | MLP        | 8.2 K  

Overwrote '(m004)-(gru)_decoder_1.1' from the log path c:\Aryan_Savant\Thesis_Projects\my_work\AFD_thesis\topology_estimation\logs\mass_sp_dm\M004\scene_1\decoder\train\etypes=1\g=m004\D=gru\(m004)-(gru)_decoder_1.1.
Model parameters saved to c:\Aryan_Savant\Thesis_Projects\my_work\AFD_thesis\topology_estimation\logs\mass_sp_dm\M004\scene_1\decoder\train\etypes=1\g=m004\D=gru\(m004)-(gru)_decoder_1.1.


c:\Anaconda3\envs\afd_env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
c:\Anaconda3\envs\afd_env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.



Initializing input processors for decoder model...

>> Domain transformer initialized for 'time' domain

>> No raw data normalization is applied

>> No feature normalization is applied

>> No time feature extraction is applied

>> No feature reduction is applied

---------------------------------------------------------------------------
Epoch 0: 100%|██████████| 16/16 [00:11<00:00,  1.37it/s, v_num=_1.1, val_loss=173.0, train_loss=223.0]
Epoch 1/5 completed
Epoch 1: 100%|██████████| 16/16 [00:10<00:00,  1.47it/s, v_num=_1.1, val_loss=110.0, train_loss=164.0]
Epoch 2/5 completed
Epoch 2: 100%|██████████| 16/16 [00:10<00:00,  1.48it/s, v_num=_1.1, val_loss=45.20, train_loss=104.0]
Epoch 3/5 completed
Epoch 3: 100%|██████████| 16/16 [00:11<00:00,  1.44it/s, v_num=_1.1, val_loss=57.60, train_loss=41.10]
Epoch 4/5 completed
Epoch 4: 100%|██████████| 16/16 [00:11<00:00,  1.44it/s, v_num=_1.1, val_loss=-13.7, train_loss=-10.7]

`Trainer.fit` stopped: `max_epochs=5` reached.



Epoch 5/5 completed
Epoch 4: 100%|██████████| 16/16 [00:11<00:00,  1.43it/s, v_num=_1.1, val_loss=-13.7, train_loss=-10.7]

Training completed in 59.04 seconds or 0.98 minutes or 0.016399316522810195 hours.

Training completed for model '(m004)-(gru)_decoder_1.1'. Trained model saved at c:\Aryan_Savant\Thesis_Projects\my_work\AFD_thesis\topology_estimation\logs\mass_sp_dm\M004\scene_1\decoder\train\etypes=1\g=m004\D=gru\(m004)-(gru)_decoder_1.1\checkpoints

---------------------------------------------------------------------------

<<<<<<<<<<<< TRAINING LOSS PLOT (TRAIN + VAL) >>>>>>>>>>>>

Creating training loss plot for (m004)-(gru)_decoder_1.1...

Training loss (train + val) plot logged at c:\Aryan_Savant\Thesis_Projects\my_work\AFD_thesis\topology_estimation\logs\mass_sp_dm\M004\scene_1\decoder\train\etypes=1\g=m004\D=gru\(m004)-(gru)_decoder_1.1


<<<<<<<<<<<< DECODER OUTPUT PLOT (TRAIN)>>>>>>>>>>>>

Creating decoder output plot for rep '1,001.378' for (m004)-(gru)_decoder_1.1..

In [16]:
from topology_estimation.settings.manager import get_checkpoint_path

trained_decoder_model = Decoder.load_from_checkpoint(get_checkpoint_path(train_log_path))


.ckpt_files available in c:\Aryan_Savant\Thesis_Projects\my_work\AFD_thesis\topology_estimation\logs\mass_sp_dm\M004\scene_1\decoder\train\etypes=1\(mass_1+mass_2+mass_3+mass_4)\D=gru\(gru)_decoder_1.1\checkpoints:

['epoch=4-step=80.ckpt']


In [18]:
trained_decoder_model.n_dims

3

#### Plot and upload

In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import os

event_dir = os.path.join('model_logs', 'trials', 'nri_model_trial3', 'version_0')
print(event_dir)

ea = event_accumulator.EventAccumulator(event_dir)
ea.Reload()

# List all tags
# print(ea.Tags())

loss_events = ea.Scalars('train_loss')
losses = [event.value for event in loss_events]
steps = [event.step for event in loss_events]

print("Steps:", steps)
print("Losses:", losses)



In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import io
from pytorch_lightning.loggers import TensorBoardLogger
from PIL import Image
import numpy as np

fig, ax = plt.subplots()
ax.plot(steps, losses, label='Train Loss')
ax.set_xlabel('Steps')
ax.set_ylabel('Loss')
ax.legend()

# Convert matplotlib fig to image
buf = io.BytesIO()
fig.savefig(buf, format='png', dpi=1000)
buf.seek(0)
image = Image.open(buf)
image_np = np.array(image)

writer = SummaryWriter("model_logs\\trials\\nri_model_trial3")

writer.add_image("test_plot", image_np.transpose(2, 0, 1), global_step=0)
buf.close()
plt.close(fig)

### Trying the config file stuff

In [1]:
import sys
import os
import torch

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from topology_estimation.config import TrainNRIConfig, SelectTopologyEstimatorModel
from data.config import DataConfig
from data.prep import load_spring_particle_data

run_type = 'train'  # or 'predict'
tp_config = TrainNRIConfig()
data_config = DataConfig()

tp_config.set_encoder_params()
tp_config.set_decoder_params()

# if tp_config.is_sparsifier:
#     tp_config.get_sparsif_config()
    
# load data
data_config = DataConfig()
data_config.set_train_dataset()

# get node and edge dataset path from which data will be loaded
node_ds_paths, edge_ds_paths = data_config.get_dataset_paths()

# load datalaoders
train_loader, valid_loader, test_loader, data_stats = load_spring_particle_data(node_ds_paths, edge_ds_paths)

dataiter = iter(train_loader)
data = next(dataiter)

n_nodes = data[0].shape[1]
n_timesteps = data[0].shape[2]
n_dims = data[0].shape[3]

ModuleNotFoundError: No module named 'augment'

In [2]:
import os
from data.config import DataConfig
from topology_estimation.config import TrainNRIConfig

data_config = DataConfig()
data_config.set_train_dataset()

tp_config = TrainNRIConfig()
tp_config.set_encoder_params()
tp_config.set_decoder_params()

log_path_nri = tp_config.get_train_log_path(500, 1)
print(log_path_nri)

if log_path_nri is not None:
    os.makedirs(log_path_nri, exist_ok=True)


'edge_estimator_2.1' already exists in the log path 'c:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\train\etypes=2\edge_estimator_2.1'.
Overwrote 'edge_estimator_2.1' from the log path c:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\train\etypes=2\edge_estimator_2.1.
c:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\train\etypes=2\edge_estimator_2.1


In [4]:
tp_config.save_params()

Model parameters saved to c:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\train\etypes=2\edge_estimator_2.1.


In [7]:
tp_config.check_if_version_exists(log_path_nri)

Version 1 already exists in the log path 'logs\spring_particles\P005\scenario_1\directed_graph\enc=mlp_1-dec=gru\healthy\H1_[OG]\dp=49-dim=4-etype=2\sparsif=[knn+time]\(sparsif)=_no_fex\[enc]=freq-[dec]=time\(enc)=PCA+first_n_modes-(dec)=PCA\v1'.
Removed version 1 from the log path logs\spring_particles\P005\scenario_1\directed_graph\enc=mlp_1-dec=gru\healthy\H1_[OG]\dp=49-dim=4-etype=2\sparsif=[knn+time]\(sparsif)=_no_fex\[enc]=freq-[dec]=time\(enc)=PCA+first_n_modes-(dec)=PCA\v1.


In [8]:
print(tp_config.log_path)
os.makedirs(tp_config.log_path, exist_ok=True)

logs\spring_particles\P005\scenario_1\directed_graph\enc=mlp_1_dec=gru\dp=49\healthy\H1_[OG]\sparsif_knn\(sparsif)_no_fex\(nri)_no_fex\v2


In [6]:
print(tp_config.fex_configs_sparsif)
print(tp_config.fex_configs_encoder)

[{'type': 'first_n_modes', 'n_modes': 5}, {'type': 'lucas', 'weight': 0.5, 'height': 9, 'age': 20}]
[{'type': 'first_n_modes', 'n_modes': 69}]


In [2]:
from topology_estimation.config import SelectTopologyEstimatorModel

model_selector = SelectTopologyEstimatorModel(
                                              framework='directed_graph')

ckpt = model_selector.select_ckpt_and_params()
print(f"Selected checkpoint: {ckpt}")

bearing
└── cwru
    └── scene_1
        └── directed_graph
            ├── <n_edge_types>
            └── etypes=2
                ├── <ds_type>
                └── OK_NOK
                    ├── <ds_subtype>
                    └── 0_N[OG--gau_m=0.1s=0.2--gau_m=0.2s=0.3]_+_0_B-007[gau_m=0.0s=0.1--gau_m=0.1s=0.2]_+_0_B-021
                        [OG]
                        ├── <model>
                        └── E=mlp_1-D=gru
                            ├── <ds_stats>
                            └── T500_m=[acc]
                                ├── <sparsif_type>
                                └── spf=_no_spf
                                    ├── <domain>
                                    └── [E]=freq-[D]=time
                                        ├── <nri_fex_type>
                                        └── (E)=_no_fex-(D)=[first_n_modes]
                                            ├── <shape_compatibility>
                                            └── E(comps)=500-D(dims)=1
                                                ├── <versions>
                                                └── v1 [0]


Available version paths:
0: logs/bearing\cwru\scene_1\directed_graph\etypes=2\OK_NOK\0_N[OG--gau_m=0.1s=0.2--gau_m=0.2s=0.3]_+_0_B-007[gau_m=0.0s=0.1--gau_m=0.1s=0.2]_+_0_B-021[OG]\E=mlp_1-D=gru\T500_m=[acc]\spf=_no_spf\[E]=freq-[D]=time\(E)=_no_fex-(D)=[first_n_modes]\E(comps)=500-D(dims)=1\v1


ValueError: invalid literal for int() with base 10: ''

predict and custom test

In [1]:
import sys
import os
import torch

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [6]:
import os
from data.config import DataConfig
from topology_estimation.config import PredictNRIConfig

data_config = DataConfig()
data_config.set_predict_dataset()

tp_config = PredictNRIConfig()

log_path_nri = tp_config.get_predict_log_path()
print(log_path_nri)

if log_path_nri is not None:
    os.makedirs(log_path_nri, exist_ok=True)

tp_config.save_predict_params()


predict number 2 for already exists for model_2.2 in the log path 'C:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\predict\etypes=2\model_2.2\predict_2.2'.
C:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\predict\etypes=2\model_2.2\predict_2.6
Predict parameters saved to C:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\predict\etypes=2\model_2.2\predict_2.6.


In [3]:
import os
from data.config import DataConfig
from topology_estimation.config import PredictNRIConfig

data_config = DataConfig()
data_config.set_custom_test_dataset()

tp_config = PredictNRIConfig()

log_path_nri = tp_config.get_custom_test_log_path()
print(log_path_nri)

if log_path_nri is not None:
    os.makedirs(log_path_nri, exist_ok=True)

tp_config.save_custom_test_params()


custom_test number 2 for already exists for model_2.2 in the log path 'C:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\custom_test\etypes=2\model_2.2\custom_test_2.2'.
C:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\custom_test\etypes=2\model_2.2\custom_test_2.5
Custom test parameters saved to C:\AFD\topology_estimation\logs\bearing\cwru\scene_1\directed_graph\custom_test\etypes=2\model_2.2\custom_test_2.5.
